In [1]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [2]:
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import os
import keras
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:

# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)


# Encode text values to a single dummy variable.  The new columns (which do not replace the old) will have a 1
# at every location where the original column (name) matches each of the target_values.  One column is added for
# each target value.
def encode_text_single_dummy(df, name, target_values):
    for tv in target_values:
        l = list(df[name].astype(str))
        l = [1 if str(x) == str(tv) else 0 for x in l]
        name2 = "{}-{}".format(name, tv)
        df[name2] = l


# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = np.asarray(df[name], dtype = np.float).mean()

    if sd is None:
        sd = np.asarray(df[name], dtype = np.float).std()

    df[name] = (np.asarray(df[name], dtype = np.float) - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        # Regression
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low

In [4]:

# This file is a CSV, just no CSV extension or headers
# Download from: http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html
df_ = pd.read_csv("./train_70%.csv")
df_test = pd.read_csv("./test_30%.csv")
print("Read df_ {} rows.".format(len(df_)))
print("Read df_test {} rows.".format(len(df_test)))
#print("Read {} rows.".format(len(df1)))
# df = df.sample(frac=0.1, replace=False) # Uncomment this line to sample only 10% of the dataset
df_ = df_.drop(df_.columns[0], axis=1)
df_test = df_test.drop(df_test.columns[0], axis=1)

df_.dropna(inplace=True,axis=1) # For now, just drop NA's (rows with missing values)
df_test.dropna(inplace=True,axis=1)

Read df_ 752285 rows.
Read df_test 322409 rows.


In [5]:
df_.head(5)

,remote_ip2num,is_tcp,is_udp,pkt_len,outcome
0,3419258414,1,0,1300,bili
1,1747765669,0,1,906,skype
2,3225463870,0,1,147,LoL
3,1747765669,0,1,320,skype
4,3225463870,0,1,795,LoL


In [6]:
df_.head(5)

,remote_ip2num,is_tcp,is_udp,pkt_len,outcome
0,3419258414,1,0,1300,bili
1,1747765669,0,1,906,skype
2,3225463870,0,1,147,LoL
3,1747765669,0,1,320,skype
4,3225463870,0,1,795,LoL


In [7]:
# # # Now encode the feature vector

# # # encode_text_dummy(df, 'protocol_type')
# # encode_numeric_zscore(df, 'forw_byte')
# # encode_numeric_zscore(df, 'back_byte')
# # encode_numeric_zscore(df, 'tot_byte')

# encode_numeric_zscore(df_, ' Flow Duration')
# encode_numeric_zscore(df_, ' Total Fwd Packets')
# encode_numeric_zscore(df_, ' Total Backward Packets')
# encode_numeric_zscore(df_, 'Total Length of Fwd Packets')
# encode_numeric_zscore(df_, ' Total Length of Bwd Packets')
# encode_numeric_zscore(df_, ' Fwd Packet Length Max')
# encode_numeric_zscore(df_, ' Fwd Packet Length Min')
# encode_numeric_zscore(df_, ' Fwd Packet Length Mean')
# encode_numeric_zscore(df_, ' Fwd Packet Length Std')
# encode_numeric_zscore(df_, 'Bwd Packet Length Max')
# encode_numeric_zscore(df_, ' Bwd Packet Length Min')
# encode_numeric_zscore(df_, ' Bwd Packet Length Mean')
# encode_numeric_zscore(df_, ' Bwd Packet Length Std')
# encode_numeric_zscore(df_, ' Min Packet Length')
# encode_numeric_zscore(df_, ' Max Packet Length')
# encode_numeric_zscore(df_, ' Packet Length Mean')
# encode_numeric_zscore(df_, ' Packet Length Std')
# encode_numeric_zscore(df_, 'total pkt count')
# encode_numeric_zscore(df_, 'total byte count')

# encode_numeric_zscore(df_test, ' Flow Duration')
# encode_numeric_zscore(df_test, ' Total Fwd Packets')
# encode_numeric_zscore(df_test, ' Total Backward Packets')
# encode_numeric_zscore(df_test, 'Total Length of Fwd Packets')
# encode_numeric_zscore(df_test, ' Total Length of Bwd Packets')
# encode_numeric_zscore(df_test, ' Fwd Packet Length Max')
# encode_numeric_zscore(df_test, ' Fwd Packet Length Min')
# encode_numeric_zscore(df_test, ' Fwd Packet Length Mean')
# encode_numeric_zscore(df_test, ' Fwd Packet Length Std')
# encode_numeric_zscore(df_test, 'Bwd Packet Length Max')
# encode_numeric_zscore(df_test, ' Bwd Packet Length Min')
# encode_numeric_zscore(df_test, ' Bwd Packet Length Mean')
# encode_numeric_zscore(df_test, ' Bwd Packet Length Std')
# encode_numeric_zscore(df_test, ' Min Packet Length')
# encode_numeric_zscore(df_test, ' Max Packet Length')
# encode_numeric_zscore(df_test, ' Packet Length Mean')
# encode_numeric_zscore(df_test, ' Packet Length Std')
# encode_numeric_zscore(df_test, 'total pkt count')
# encode_numeric_zscore(df_test, 'total byte count')

# encode_text_index(df_, ' Label')
# encode_text_index(df_test, ' Label')
# # num_classes = len(outcomes)

# # # display 5 rows

# # df.dropna(inplace=True,axis=1)
# # df[0:5]
# # # This is the numeric feature vector, as it goes to the neural net

In [8]:
encode_numeric_zscore(df_, 'remote_ip2num')
# encode_numeric_zscore(df_, 'is_tcp')
# encode_numeric_zscore(df_, 'is_udp')
encode_numeric_zscore(df_, 'pkt_len')
encode_text_index(df_, 'outcome')
encode_numeric_zscore(df_test, 'remote_ip2num')
encode_numeric_zscore(df_test, 'pkt_len')
encode_text_index(df_test, 'outcome')

array(['LoL', 'bili', 'download', 'netease', 'skype', 'tencent', 'youtube'], dtype=object)

In [9]:
x_train, y_train = to_xy(df_, 'outcome')
x_test, y_test = to_xy(df_test, 'outcome')
# x, y = to_xy(df_, ' Label')

In [10]:
df_.head(5)

,remote_ip2num,is_tcp,is_udp,pkt_len,outcome
0,1.587349,1,0,0.910912,1
1,0.102173,0,1,0.163852,4
2,1.415156,0,1,-1.275280,0
3,0.102173,0,1,-0.947256,4
4,1.415156,0,1,-0.046614,0


In [11]:
x_test

array([[ 1.14037454,  1.        ,  0.        ,  0.88518053],
       [ 0.10253431,  0.        ,  1.        , -0.26666588],
       [ 0.10253431,  0.        ,  1.        , -0.23061138],
       ..., 
       [-1.18292356,  1.        ,  0.        ,  0.88518053],
       [-1.18292356,  1.        ,  0.        ,  0.88518053],
       [ 1.14037454,  1.        ,  0.        ,  0.88518053]], dtype=float32)

In [12]:
import keras.backend as K
K.clear_session()

In [13]:
# Create neural net
model = Sequential()

# Used relu for activation function
# model.add(Dense(4, input_dim=x_train.shape[1], kernel_initializer='normal', activation='relu'))
# model.add(Dense(8, input_dim=x_train.shape[1], kernel_initializer='normal', activation='relu'))
# model.add(Dense(4, input_dim=x_train.shape[1], kernel_initializer='normal', activation='relu'))

model.add(Dense(3, input_dim=x_train.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(8, input_dim=x_train.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(y_train.shape[1],activation='sigmoid'))

In [14]:
print(x_train.shape[1])

4


In [15]:
# Print the model summary
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 3)                 15        
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 32        
_________________________________________________________________
dense_3 (Dense)              (None, 7)                 63        
Total params: 110
Trainable params: 110
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
batch_size = 100
model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=1,batch_size = batch_size, epochs=50)

Train on 752285 samples, validate on 322409 samples
Epoch 1/50
752285/752285 [==============================] - 26s - loss: 0.3292 - acc: 0.9008 - val_loss: 0.1451 - val_acc: 0.9619
Epoch 2/50
752285/752285 [==============================] - 30s - loss: 0.1360 - acc: 0.9631 - val_loss: 0.1319 - val_acc: 0.9791
Epoch 3/50
752285/752285 [==============================] - 26s - loss: 0.1271 - acc: 0.9647 - val_loss: 0.1250 - val_acc: 0.9799
Epoch 4/50
752285/752285 [==============================] - 24s - loss: 0.1226 - acc: 0.9667 - val_loss: 0.1177 - val_acc: 0.9807
Epoch 5/50
752285/752285 [==============================] - 24s - loss: 0.1107 - acc: 0.9707 - val_loss: 0.1087 - val_acc: 0.9670
Epoch 6/50
752285/752285 [==============================] - 25s - loss: 0.1061 - acc: 0.9745 - val_loss: 0.1045 - val_acc: 0.9830
Epoch 7/50
752285/752285 [==============================] - 26s - loss: 0.1031 - acc: 0.9778 - val_loss: 0.1022 - val_acc: 0.9833
Epoch 8/50
752285/752285 [============

In [17]:
# Measure accuracy
pred = model.predict(x_test)
pred = np.argmax(pred,axis=1)
y_eval = np.argmax(y_test,axis=1)
score = metrics.accuracy_score(y_eval, pred)
print("Validation score: {}".format(score))

Validation score: 0.9850407401778486


In [18]:
print(y_eval[:100])

[6 4 4 2 5 2 2 6 4 4 2 4 1 0 1 5 4 4 2 4 4 4 2 4 1 2 4 2 4 2 2 4 2 6 2 2 6
 2 6 4 5 0 2 5 2 2 4 0 5 2 2 2 1 2 1 2 2 4 2 5 5 2 0 2 2 4 5 4 0 4 4 6 2 0
 4 5 2 6 4 2 1 4 4 5 2 2 6 4 0 2 5 4 2 4 4 4 4 2 1 0]


In [19]:
print(pred[:100])

[6 4 4 2 5 2 2 6 4 4 2 4 1 0 1 5 4 4 2 4 4 4 2 4 1 2 4 2 4 2 2 4 2 6 2 2 6
 2 6 4 5 0 2 5 2 2 4 0 5 2 2 2 1 2 1 2 2 4 2 5 5 2 0 2 2 4 5 4 0 4 4 6 2 0
 4 5 2 6 4 2 1 4 4 5 2 2 6 4 0 2 5 4 2 4 4 4 4 2 1 0]


In [20]:
iff = np.ones(272618, dtype=int)

In [21]:
iff = 3*iff

In [22]:
from sklearn.metrics import classification_report,confusion_matrix
Cm = confusion_matrix(y_eval,pred)
C = np.sum(Cm)
Cm = Cm/C
print('Confusion Matrix:')
print(np.array_str(Cm, precision=4, suppress_small=True))

Confusion Matrix:
[[ 0.0836  0.      0.      0.      0.      0.      0.    ]
 [ 0.      0.0882  0.0002  0.      0.      0.0031  0.0003]
 [ 0.      0.      0.3265  0.      0.      0.      0.    ]
 [ 0.      0.      0.0003  0.      0.      0.0027  0.    ]
 [ 0.      0.0001  0.0009  0.      0.3394  0.0026  0.0005]
 [ 0.      0.0004  0.0003  0.      0.0001  0.0889  0.0005]
 [ 0.      0.0001  0.0011  0.      0.      0.0015  0.0585]]


In [23]:
# from sklearn.metrics import precision_recall_fscore_support
# print('DDos precision & recall & f1_score & support:')
# typ = [ "precision", "recall", "f1_score", "support"]
# ddos_prfs = precision_recall_fscore_support(y_eval, pred)
# ddos_list = [x for x,_ in ddos_prfs]
# for f, b in zip(typ, ddos_list):
#     print( f, b)

In [24]:
print('Normal precision & recall & f1_score & support:')
typ = [ "precision", "recall", "f1_score", "support"]
ddos_prfs = precision_recall_fscore_support(y_eval, pred)
ddos_list = [_ for x,_ in ddos_prfs]
for f, b in zip(typ, ddos_list):
    print( f, b)

Normal precision & recall & f1_score & support:


NameError: name 'precision_recall_fscore_support' is not defined

In [ ]:
valAcc = history_cb.val_acc
valLoss = history_cb.val_loss
epoch_it = np.arange(1,16)
plt.subplot(121)
plt.plot(epoch_it, valAcc)
plt.grid()
plt.xlabel('Epoch')
plt.ylabel('Validation accuracy')
plt.tight_layout()
plt.subplot(122)
plt.plot(epoch_it, valLoss)
plt.grid()
plt.xlabel('Epoch')
plt.ylabel('Validation loss')
plt.tight_layout()
plt.show()

In [ ]:
Ls = history_cb.losses
Acc = history_cb.accs
ntr = np.shape(x_train)[0]
epochNum = []
for i in range(5040):
    epochNum.append(i*100/ntr)

plt.subplot(122)
matplotlib.pyplot.semilogy(epochNum, Ls)
plt.grid()
plt.xlabel('Epoch')
plt.ylabel('Training Loss')
plt.tight_layout()
# plt.show()
plt.subplot(121)
matplotlib.pyplot.semilogy(epochNum, Acc)
# plt.plot(epochNum, Acc)
plt.grid()
plt.xlabel('Epoch')
plt.ylabel('Training Accuracy')
plt.tight_layout()
# plt.axis([0.3,3,0.96,1])
plt.show()

In [ ]:
matplotlib.pyplot.semilogy(epochNum, Acc)
# plt.plot(epochNum, Acc)
plt.grid()
plt.xlabel('Epoch')
plt.ylabel('Training Accuracy')
plt.tight_layout()
# plt.axis([0.3,3,0.96,1])
plt.show()



In [ ]:
from keras.models import load_model

# model.save('model_trained_nosampled_sampling@100%.h5')  # creates a HDF5 file 'my_model.h5'
# del model  # deletes the existing model

# # returns a compiled model
# # identical to the previous one
# model = load_model('my_model.h5')

In [ ]:
# hist = model.fit(x, y, validation_split=0.2)
# print(hist.history)

In [ ]:
# np.shape(hist.history['acc'])

In [ ]:
# model2 = load_model('model_trained_sampling@100%.h5')

In [ ]:
# # Measure accuracy
# pred = model2.predict(x_test)
# pred = np.argmax(pred,axis=1)
# y_eval = np.argmax(y_test,axis=1)
# score = metrics.accuracy_score(y_eval, pred)
# print("Validation score: {}".format(score))

In [ ]:
/